<center>
    <img src="https://github.com/juanandres-montero/macroeconometria/blob/main/logo%20EEC%20grande.png?raw=1" width="396" height="161"><br>
    <b style="font-size:200%">EC4301 MACROECONOMETRÍA</b><br>
    <b style="font-size:100%">Asistente: Juan Andrés Montero</b><br><br>
    <b style="font-size:100%"> Laboratorio #3:</b>
    <br>    
    <div style="font-size:150%; color:white; background-color: #0064b0; padding-top: 20px; padding-bottom: 20px; width: 50%; margin: 0 auto; border-radius: 15px;">
        MC2E, SUR y MC3E
    </div>
</center>

<br>

<i style="font-size:80%"> 
    Creado: Setiembre 2024
</i>

<hr>

En MCI hay una ecuación exactamente identificada. Otra forma de estimar parámetros de forma estructural en ecuación por ecuación con información limitada es en Mínimos Cuadrados en 2 Etapas.

# Mínimos cuadrados en dos etapas (MC2E)

Queremos estimar el siguiente modelo con los datos de la economía estadounidense (1921-1941), que importamos de internet: [TBL16.DTA](https://stats.idre.ucla.edu/stat/stata/examples/greene/TBL16-2.DTA).

$$
C_t = \alpha_0 + \alpha_1P_t + \alpha_2P_{t-1} + \alpha_3(W_t^p + W_t^g) + \epsilon_{1t} 
$$

$$
I_t = \beta_0 + \beta_1P_t + \beta_2P_{t-1} + \beta_3K_{t-1} + \epsilon_{2t}
$$

$$
W_t^p = \gamma_0 + \gamma_1X_t + \gamma_2X_{t-1} + \gamma_3A_t + \epsilon_{3t}
$$

$$
X_t = C_t + I_t + G_t 
$$

$$
P_t = X_t - T_t - W_t^p 
$$

$$
K_t = K_{t-1} + I_t 
$$



| Símbolo  | Descripción                                             |
|----------|---------------------------------------------------------|
| $C_t$    | Consumo                                                 |
| $I_t$    | Inversión                                               |
| $W_t^p$  | Salarios del sector privado                             |
| $W_t^g$  | Salarios del sector público                             |
| $X_t$    | Demanda agregada                                        |
| $P_t$    | Beneficios privados                                     |
| $K_t$    | Acervo de capital                                       |
| $T_t$    | Impuestos                                               |
| $G_t$    | Gasto gubernamental sin contar salarios                 |
| $A_t$    | Tendencia lineal centrada en 1931                       |

Este modelo es conocido como Klein I, y es bastante representativo de los grandes modelos keynesianos de ecuaciones simultáneas que se utilizaban para estudiar el comportamiento macroeconómico antes de la crítica de Lucas.

Se va a utilizar el supuesto de que las variables endógenas son $C_t$, $I_t$ y $W_t^p$. Y el resto serán exógenas

Incluimos las identidades contables al modelo y creamos $A_t$ junto con $W_t^p + W_t^g$:

```generate w = wg+wp```
```generate k = kl+i```
```generate A=year-1931```

Creamos las series rezagadas
```generate pl = p[_n-1]```
```generate xl = x[_n-1]```


## Estimar MC2E 

1. Se escribe el comando ```reg3```, 
2. Se escriben las ecuaciones estructurales del modelo
4. Se establece que se va a usar MC2E escribiendo ```2sls```
5. Se indica cuáles son las variables exógenas del sistema con ```inst```:

```reg3 (c p p1 w) (1 p p1 k1) (wp x x1 A), 2sls inst(t wg g A p1 x1 k1)```

### ¿Y la identificación?

El problema de identificación no es puramente estadístico, sino que se trata de resolver un sistema de ecuaciones, específicamente de encontrar parámetros estructurales a partir de los parámetros reducidos. Para que este sistema tenga solución, necesitamos imponer restricciones adicionales. Una manera de hacerlo es excluyendo ciertas variables exógenas de una ecuación pero no de otra, lo que equivale a decir que el parámetro correspondiente es 0 en la ecuación de donde se excluye. Sin embargo, hay otras formas de imponer restricciones. 

## **Porqué usar MC2E?** 
- Corregir endogeneidad
    - Etapa 1: Regresión en forma reducida
    - Etapa 2: MCO a la ecuación transformada

# Regresiones aparentemente no relacionadas (SUR)

Considere el siguiente modelo SUR (Greene, 2002, p.340) con los datos del archivo ```Grunfeld and Griliches (1960).xlsx```

$$
I_{it} = \beta_{1i} + \beta_{2i}F_{it} + \beta_{3i}C_{it} + \epsilon_{it}
$$

| Símbolo  | Descripción                                                                                                                                       |
|----------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| $I_{it}$ | Inversión realizada por la firma $i$ en el periodo $t$, incluyendo gastos en planta, equipo, mantenimiento y reparaciones.                         |
| $C_{it}$ | Valor del stock de capital de planta y equipo con el que la firma $i$ comienza el periodo $t$.                                                     |
| $F_{it}$ | Valor de mercado de la empresa $i$ al inicio del periodo $t$.                                                                                      |

### Firmas
- $i \in \{1,2,3,4,5\}$, donde:
  - Firma 1: General Motors
  - Firma 2: Chrysler
  - Firma 3: General Electric
  - Firma 4: Westinghouse
  - Firma 5: US Steel

### Periodo de Datos
- 1935-1954.

Para estimar un modelo SUR, el método más eficiente es el de mínimos cuadrados generalizados factibles (MCGF). Aunque las ecuaciones parecen no estar relacionadas porque cada una tiene su propia variable dependiente, los errores entre las ecuaciones pueden estar correlacionados. El estimador MCG corrige esta correlación, haciendo que sea más eficiente que MCO. 
Para implementar MCGF, utilizamos el comando ```sureg``` , y al comparar con la estimación por MCO, los errores estándar suelen ser menores:
```
*Estimamos el modelo SUR con FGLS.
sureg (invest1 market1 stock1)  ///
      (invest2 market2 stock2)  ///
      (invest3 market3 stock3)  ///
      (invest4 market4 stock4)  ///
      (invest5 market5 stock5)
```

Podemos comparar estos resultados con MCO ecuación por ecuación:
```
reg invest1 market1 stock1
reg invest2 market2 stock2
reg invest3 market3 stock3
reg invest4 market4 stock4
reg invest5 market5 stock5
```


Note que los errores estándar de MCO ecuación por ecuación en general son mayores que los de 
MCGF, recuerde que este segundo es una versión del primero, pero corregido por la 
covarianza de los errores entre ecuaciones.

Podemos poner a prueba la hipótesis 
$$ \beta_{j1} = \beta_{j2} = \beta_{j3} = \beta_{j4} = \beta_{j5} \text{ para } j=1,2,3 $$
es decir, que los tres coeficientes de interés son iguales para las 5 firmas. Lo haremos mediante 
una prueba de razón de verosimilitud. Para ello, primero estimamos el modelo sin restricciones:

```
sureg (invest1 market1 stock1)  ///
      (invest2 market2 stock2)  ///
      (invest3 market3 stock3)  ///
      (invest4 market4 stock4)  ///
      (invest5 market5 stock5), isure
```

Queremos iterarla porque 
> "los estimadores de máxima verosimilitud se pueden obtener iterando el procedimiento MCGF" (Greene, 2002, p. 
347)."

Luego escribimos ```ereturn list```, que almacena todos los resultados de interés de nuestra estimación: en particular, en ```e(11)``` almacena la log-verosimilitud (ln $L_u$): -459.0922.


Ahora queremos usar el modelo con restricciones:

$$ I_{it} = \beta_1 + \beta_2 F_{it} + \beta_3 C_{it} + \epsilon_{it} \quad \forall i \in \{1,2,3,4,5\} $$


Primero, necesitamos indicarle a STATA que queremos estudiar las variables de inversión, valor de mercado y stock de capital para todas las firmas. Lo más sencillo es reorganizar los datos para que ```investj```, ```marketj``` y ```stockj``` (donde $j=1,2,3,4,5$) se agrupen en una sola variable cada una: ```invest```, ```market``` y ```stock```, con una columna adicional de identificador, por ejemplo, ```firm```.

Esto se logra usando el comando ```reshape long```, que transforma los datos de un formato horizontal (```wide```) a uno vertical (```long```). Simplemente indicamos las variables de interés, especificamos que deben ser diferenciadas en el tiempo con ```i(time)```, y que STATA debe crear una variable índice con ```j(firm)```. Es crucial que las variables originales estuvieran nombradas como ```investj```, ```marketj``` y ```stockj```, para que STATA entienda cómo usar “j” como identificador.


Una vez hicimos esto, solo debemos establecer quiénes son nuestros agentes y cuál variable 
indica el tiempo; eso es lo que hace ```xtset firm time```.

Con solo ```xtgls``` se estimaría el modelo con MCGF, pero agregamos ```igls``` para iterar y así 
obtener la log-verosimilitud (ln $L_r$): -515.4222 — ```panels(correlated)``` solo le indica a STATA que 
los errores entre empresas están correlacionados.

Estos resultados bastan para calcular el estadístico de razón de verosimilitudes

$$ -2(\ln L_r - \ln L_u) = -2(-515.4222 - -459.0922) = 112.66 $$

Solamente restaría calcular el valor crítico para concluir sobre nuestra hipótesis de igualdad 
de coeficientes. Recuerde que el estadístico de la prueba de razón de verosimilitudes se distribuye 
como $\chi^2$ con una cantidad de grados de libertad igual al número de restricciones (12, en este 
caso). Para una confianza del 99\% escribimos ```display invchi2(12, 0.99)``` y obtenemos un 
valor crítico de 26.127; que es menor que el valor de nuestro estadístico: se rechaza nuestra 
hipótesis nula para todos los niveles de significancia convencionales.

## ¿Y “la macro”?

> En macroeconomía, a veces se construyen agentes representativos para simplificar el estudio de los agregados económicos. Grunfeld y Griliches (1960) analizaron esta metodología para la inversión y encontraron que su modelo agregado de inversión ofrecía un mejor ajuste que las regresiones individuales de cada empresa. Argumentaron que, aunque los investigadores no pueden conocer a fondo a cada individuo, observar el conjunto de agentes permite superar esta limitación y estudiar el agregado de manera más efectiva.


Es importante señalar que gran parte del análisis de Grunfeld y Griliches se basaba en estimaciones por Mínimos Cuadrados Ordinarios (MCO). Hoy en día, se recomienda usar Mínimos Cuadrados Generalizados (MCG), que ofrece una fórmula de bondad de ajuste diferente, detallada en Greene (2002, p. 345).

## ¿Por qué SUR?
- Todas las variables son exógenas.
- Existe una correlación contemporánea entre los errores de la ecuación.

# Mínimos cuadrados en tres etapas (MC3E)

Ahora nos interesa estimar el modelo Klein por MC3E; de nuevo queremos hacerlo con los 
datos de la economía estadounidense entre 1921 y 1941. Después de los ajustes preliminares que 
hicimos arriba, solo necesitamos correr lo siguiente:

```reg3 (c p p1 w) (i p p1 k1) (wp x x1 A), 3sls inst(t wg g A p1 x1 k1)```

El código para estimar el modelo con MC3E es casi idéntico al que usamos para MC2E. Las primeras dos etapas de MC3E son similares a las de MC2E. En MC3E, escribimos las tres ecuaciones estructurales a la vez porque utilizamos un método de información completa, y la tercera etapa es una corrección de la varianza con MCGF. En nuestros resultados (que omitimos por espacio), observamos que los errores estándar suelen ser menores en comparación con MC2E.

## ¿Por qué MC3E?

- Alguna de las variables explicativas son endógenas.
- Existe una correlación contemporánea entre los errores de la ecuación.